Fine-tuning DistilBERT: full model, 4 epochs with a learning rate of 5e-6 and weight decay of 0.01 and a batch size 32. Data were truncated at 100 tokens.

# Install

In [ ]:
!pip install -U -q datasets transformers torchinfo

     |████████████████████████████████| 441 kB 35.9 MB/s 
     |████████████████████████████████| 5.3 MB 59.8 MB/s 
     |████████████████████████████████| 163 kB 69.5 MB/s 
     |████████████████████████████████| 115 kB 57.8 MB/s 
     |████████████████████████████████| 212 kB 45.9 MB/s 
     |████████████████████████████████| 127 kB 56.6 MB/s 
     |████████████████████████████████| 7.6 MB 48.2 MB/s 
     |████████████████████████████████| 115 kB 56.3 MB/s 


# Imports

In [ ]:
from datasets import Dataset, DatasetDict, ClassLabel, Value, load_dataset, load_metric
from huggingface_hub import notebook_login
from torchinfo import summary
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding

# Load Data

In [ ]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
ds = load_dataset('sara-nabhani/lfd-proj',
    'csv', 
    data_files={'train': 'train.csv', 'val': 'val.csv', 'test': 'test.csv'}
)

cl = ClassLabel(names=list(ds['train'].unique('label')))
ds = ds.cast_column('label', cl)

ds

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/sara-nabhani___csv/sara-nabhani--lfd-proj-a35fbd5b9bbbc3d0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Casting the dataset:   0%|          | 0/2 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 12240
    })
    val: Dataset({
        features: ['tweet', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 860
    })
})

In [ ]:
ds['train'].features

{'tweet': Value(dtype='string', id=None),
 'label': ClassLabel(names=['OFF', 'NOT'], id=None)}

In [ ]:
ds['test'].features

{'tweet': Value(dtype='string', id=None),
 'label': ClassLabel(names=['OFF', 'NOT'], id=None)}

In [ ]:
ds['val'].features

{'tweet': Value(dtype='string', id=None),
 'label': ClassLabel(names=['OFF', 'NOT'], id=None)}

In [ ]:
import torch
torch.cuda.is_available()

True

# Preprocessing

In [ ]:
model_id = 'distilbert-base-cased'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(examples):
    return tokenizer(examples['tweet'], padding='max_length', truncation=True, max_length=100)

tokenized_ds = ds.map(tokenize_function, batched=True)

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

# load model and retrain

In [ ]:
metric = load_metric("accuracy")
# evaluation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
training_args = TrainingArguments(    
  output_dir="./results",
  report_to='all',
  learning_rate=5e-6,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=32,
  num_train_epochs=4,
  weight_decay=0.01,
  no_cuda=False,
  # bf16=IS_CUDA_AVAILABLE,
  fp16=True,
  evaluation_strategy='epoch',
  logging_strategy='epoch',
)
  
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_ds['train'],
  eval_dataset=tokenized_ds['val'],
  compute_metrics=compute_metrics,
  tokenizer=tokenizer,
  data_collator=data_collator,
)
trainer.train()



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


Downloading:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weigh

Epoch,Training Loss,Validation Loss,Accuracy
1,0.513600,0.457731,0.787000
2,0.428300,0.451268,0.794000
3,0.395700,0.457930,0.798000
4,0.372000,0.456676,0.802000


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_

TrainOutput(global_step=3060, training_loss=0.4274321213267208, metrics={'train_runtime': 246.414, 'train_samples_per_second': 198.69, 'train_steps_per_second': 12.418, 'total_flos': 1266719499648000.0, 'train_loss': 0.4274321213267208, 'epoch': 4.0})

# Evaluation


In [ ]:
print("Prediction on validation data:")
print(trainer.predict(tokenized_ds['val'])[-1])

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 32


Prediction on validation data:


{'test_loss': 0.4566757082939148, 'test_accuracy': 0.802, 'test_runtime': 1.0948, 'test_samples_per_second': 913.39, 'test_steps_per_second': 29.228}


In [ ]:
print("Prediction on test data:")
print(trainer.predict(tokenized_ds['test'])[-1])

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 860
  Batch size = 32


Prediction on test data:


{'test_loss': 0.3859889507293701, 'test_accuracy': 0.8395348837209302, 'test_runtime': 0.9526, 'test_samples_per_second': 902.795, 'test_steps_per_second': 28.344}


# Model summary

In [ ]:
print("model config")
print(model.config)
print("model summary")
print(summary(model))

model config
DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "vocab_size": 28996
}

model summary
Layer (type:depth-idx)                                  Param #
DistilBertForSequenceClassification                     --
├─DistilBertModel: 1-1                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              22,268,928
│    │    └

# Pushing and saving output

In [ ]:
id2label = {cl.str2int(i): i for i in cl.names}
label2id = {v: k for k, v in id2label.items()}
id2label

{0: 'OFF', 1: 'NOT'}

In [ ]:
model.config.update({
    'id2label': id2label,
    'label2id': label2id,
})
model.config

DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "OFF",
    "1": "NOT"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NOT": 1,
    "OFF": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "vocab_size": 28996
}

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.create_repo(repo_id="distilbert-base-cased-finetuned-2", private=True)

'https://huggingface.co/sara-nabhani/distilbert-base-cased-fintuned-2'

In [ ]:
hub_name = 'sara-nabhani/distilbert-base-cased-finetuned-2'
tokenizer.push_to_hub(hub_name)
model.push_to_hub(hub_name)

tokenizer config file saved in /tmp/tmpsy9o5afv/tokenizer_config.json
Special tokens file saved in /tmp/tmpsy9o5afv/special_tokens_map.json
Uploading the following files to sara-nabhani/distilbert-base-cased-fintuned-2: tokenizer.json,tokenizer_config.json,special_tokens_map.json,vocab.txt
Configuration saved in /tmp/tmpvrxfocw1/config.json
Model weights saved in /tmp/tmpvrxfocw1/pytorch_model.bin
Uploading the following files to sara-nabhani/distilbert-base-cased-fintuned-2: config.json,pytorch_model.bin


CommitInfo(commit_url='https://huggingface.co/sara-nabhani/distilbert-base-cased-fintuned-2/commit/ed1c985b7586037bdcca2630aaa1f384786c8e80', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='ed1c985b7586037bdcca2630aaa1f384786c8e80', pr_url=None, pr_revision=None, pr_num=None)